<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [89]:
#!pip install imbalanced-learn -q
#!pip install wordcloud
#!pip install swifter - q
#!pip install xgboost
#!pip install urlextract

In [732]:
import pandas as pd
from tqdm import notebook
import re
from time import time 
import torch
import swifter
import transformers
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np
from numpy.random import default_rng
import transformers
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report, make_scorer
from __future__ import unicode_literals, print_function
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import FixedThresholdClassifier, TunedThresholdClassifierCV
from sklearn.feature_extraction.text import CountVectorizer

In [88]:
from urlextract import URLExtract

In [3]:
#from wordcloud import WordCloud

In [4]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LeeCo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LeeCo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [17]:
pd.set_option('display.max_colwidth', None)

In [5]:
RANDOM_STATE = 88

## Подготовка

In [6]:
url = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

In [39]:
df = pd.read_csv(url)

In [36]:
df.head()

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [10]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [11]:
df['text'].duplicated().sum()

np.int64(0)

In [12]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [12]:
def w_count(dataframe, df_cleaning=False, stop_words_filtered = False):

    if df_cleaning == True:
        dataframe = dataframe.str.lower()
        display('text lowered')
        dataframe = dataframe.swifter.apply(lambda x: re.sub(r'\W', ' ', x))
        display('only words in text ')
        dataframe = dataframe.swifter.apply(lambda x: re.sub(r'\s+', ' ', x))
        display('text is striped')
        
    
    word2count = {}
    
    for data in dataframe:
        words = nltk.word_tokenize(data)
        for word in words:
            if word not in word2count:
                word2count[word] = 1
            else:
                word2count[word] += 1

    if stop_words_filtered == True:
        display('stop words are removed')
        stop_words = set(stopwords.words('english'))
        filtered_word2count = {word: count for word, count in word2count.items() if word not in stop_words}
        word_freq_df = pd.DataFrame(list(filtered_word2count.items()), columns=['Word', 'Frequency'])
        word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False)
    else:
        filtered_word2count = {word: count for word, count in word2count.items()}
        word_freq_df = pd.DataFrame(list(filtered_word2count.items()), columns=['Word', 'Frequency'])
        word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False)

    return word_freq_df

In [15]:
######################################################################################

In [61]:
df['len'] = df['text'].swifter.apply(lambda x: len(x))

Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

In [183]:
#df[df['len']>2500]['text'].sample()

In [184]:
#print(' '.join([clean_text(i) for i in nl_split(df['text'][3536])]))

In [128]:
#print(clean_text(df['text'][4581]))

In [127]:
#nl_split(df['text'][152428])

In [63]:
#nl_split(df['text'][4581])

In [639]:
def url_rem(string_):
    extractor = URLExtract()
    urls = extractor.find_urls(string_)
    for url in urls:
        string_ = re.sub(re.escape(url), ' xxlink ', string_)
    
    string_ = re.sub(r'https?://\S+|www\.\S+', ' xxlink ', string_)
    string_ = re.sub(r'<.*?>+', ' xxlink ', string_)
    return string_

In [640]:
 url_rem('(taken from: http://ec.europa.eu/justice_home/news/events/events_2008_en.html) \n\nEuropean')

'(taken from:  xxlink  \n\nEuropean'

In [594]:
debug_samp = df['text'].sample(50)

debug_samp.apply(clean_sents)


PatternError: unbalanced parenthesis at position 63

In [641]:
debug_samp.apply(clean_sents)

97927                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
urlparse

In [630]:
[url_rem(sen) for sen in nl_split(df['text'][149325])]

['Please read and notice the word jointly.',
 '(taken from:  xxlink ',
 'European Hearing: ""Crimes Committed by Totalitarian Regimes""',
 'On 8 April 2008 a European Hearing entitled ""crimes committed by totalitarian regimes"" took place in Brussels.',
 'This was organised jointly by the Slovenian Presidency of the European Union and the European Commission around the two key issues of recognition of the crimes and reconciliation.   "']

In [631]:
url_rem(df['text'][149325])

'"\nPlease read and notice the word jointly. \n\n(taken from:  xxlink \n\nEuropean Hearing: ""Crimes Committed by Totalitarian Regimes""\nOn 8 April 2008 a European Hearing entitled ""crimes committed by totalitarian regimes"" took place in Brussels. This was organised jointly by the Slovenian Presidency of the European Union and the European Commission around the two key issues of recognition of the crimes and reconciliation.   "'

In [607]:
df['text'][149325][60:70]

'tp://ec.eu'

In [291]:
def mail_user_rem(string_):
    
    string_ = re.sub(r'[A-Za-z0-9\_]*@[A-Za-z]*\.?[A-Za-z0-9]*', ' xxmail ', string_)
    string_ = re.sub(r'User:\S+', ' xxuser ', string_)

    return string_

In [629]:
def nl_split(string):
    
    string = re.split('\n\n|\n',string)
    result = [sent_tokenize(i.strip()) for i in string]
    result = sum(result, [])
    
    return [i for i in result if len(i)>1]

In [318]:
def clean_text(string):
    
    string = re.sub(r'\[{1,}.+\]{1,}', ' xxbrct ', string)
    string = re.sub(r'\{{1,}.+\}{1,}', ' xxrbrct ', string)
    
    string = re.sub(r'(\W)\1+|(\D)\1+', r'\1', string)
    string = re.sub(r'[\_\+]', ' ', string)
    string = re.sub("['”‘’“]", r'"', string)
    string = re.sub(r'\s+', ' ', string)
    
    return string.strip()

In [604]:
def clean_sents(string):
    #string = url_rem(string)
    sents = nl_split(string)
    cl_sents = [url_rem(sent) for sent in sents]
    cl_sents = [mail_user_rem(sent) for sent in cl_sents]
    #string = mail_user_rem(string)
    #sents = nl_split(string)
    cl_sents = [clean_text(sent) for sent in cl_sents]
    return cl_sents

In [643]:
#clean_sents(df['text'][80726])

In [322]:
def chunker(string):

    sents_ = clean_sents(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]

    #for sent in tokenized_sents:
    #    if sent not in usents:# and sent!=[101,1012,102] and sent!=[101,102]:
    #        usents.append(sent)

    if len(tokenized_sents)>1:
        result = sum(tokenized_sents,[])
    else:
        result = tokenized_sents[0]
    
        
    if len(result)>512:

        result = result[:129] + result[-382:]+[102]
        
    
    return result

In [326]:
chunker(df['text'][80726])

[101,
 4717,
 19888,
 2080,
 1010,
 11320,
 6873,
 1010,
 1004,
 4040,
 2666,
 3104,
 2005,
 13157,
 2213,
 1004,
 12403,
 2213,
 2037,
 2743]

In [483]:
df['text'].sample(50).apply(chunker).iloc[0]

PatternError: unbalanced parenthesis at position 30

In [331]:
df['text'][:5].apply(lambda x: chunker(x)).values

array([list([101, 7526, 102, 101, 2339, 1996, 10086, 2015, 2081, 2104, 2026, 5310, 18442, 13076, 12392, 2050, 5470, 2020, 16407, 1029, 102, 101, 2027, 4694, 1000, 1056, 3158, 9305, 22556, 1010, 2074, 8503, 2006, 2070, 3806, 2044, 1045, 5444, 2012, 2047, 2259, 14421, 6904, 2278, 1012, 102, 101, 1998, 3531, 2123, 1000, 1056, 6366, 1996, 23561, 2013, 1996, 2831, 3931, 2144, 1045, 1000, 1049, 3394, 2085, 1012, 6486, 1012, 16327, 1012, 4229, 1012, 2676, 102]),
       list([101, 1040, 1000, 22091, 2860, 999, 102, 101, 2002, 3503, 2023, 4281, 6120, 1045, 1000, 1049, 9428, 5881, 2007, 1012, 102, 101, 4283, 1012, 102, 101, 1006, 2831, 1007, 2538, 1024, 4868, 1010, 2254, 2340, 1010, 2355, 1006, 11396, 1007, 102]),
       list([101, 4931, 2158, 1010, 1045, 1000, 1049, 2428, 2025, 2667, 2000, 10086, 2162, 1012, 102, 101, 2009, 1000, 1055, 2074, 2008, 2023, 3124, 2003, 7887, 9268, 7882, 2592, 1998, 3331, 2000, 2033, 2083, 10086, 2015, 2612, 1997, 2026, 2831, 3931, 1012, 102, 101, 2002, 3849, 2000, 

In [294]:
df['text'][80726]

'"\n\n Matt Crypto, Lupo, & Stormie cover for Nazism & spam their rants http://rexcurry.net/wikipedialies.html \n\nThere are some regular writers on Wikipedia who knowingly engage in personal attacks and deletions (even of links) of anyone who explains the connection between Edward Bellamy and his cousin Francis Bellamy (author of the pledge of allegiance) and National Socialism. Some of the worst falsifiers on Wikipedia are Stormie, Lupo, and Matt Crypto. They were challenged to respond to the facts that they suppressed or to concede, and they each conceded that the facts that they suppress are correct and that they delete those facts because they do not want Wikipedia readers to know the truth. They constantly spam their own point of view in their diatribes in which they try to cover up for Nazism. It is behavior that breaks Wikipedia\'s rules. What kind of people cover-up for Nazis and that horrid ideology?\n\nMany regular writers on Wikipedia are intellectually dishonest  and they 

In [16]:
df_toxic = df[df['toxic']==1]#['text']

In [17]:
df_non_toxic = df[df['toxic']==0]#['text']

In [18]:
words_w_trash = w_count(
    df['text'],
    df_cleaning=False)

In [19]:
words_w_trash0 = w_count(
    df_non_toxic['text'],
    df_cleaning=False)

In [20]:
words_w_trash1 = w_count(
    df_toxic['text'],
    df_cleaning=False)

In [21]:
words_wo_trash = w_count(
    df['text'],
    df_cleaning=True)

'text lowered'

Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

'only words in text '

Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

'text is striped'

In [22]:
words_wo_trash0 = w_count(
    df_non_toxic['text'],
    df_cleaning=True)

'text lowered'

Pandas Apply:   0%|          | 0/143106 [00:00<?, ?it/s]

'only words in text '

Pandas Apply:   0%|          | 0/143106 [00:00<?, ?it/s]

'text is striped'

In [23]:
words_wo_trash1 = w_count(
    df_toxic['text'],
    df_cleaning=True)

'text lowered'

Pandas Apply:   0%|          | 0/16186 [00:00<?, ?it/s]

'only words in text '

Pandas Apply:   0%|          | 0/16186 [00:00<?, ?it/s]

'text is striped'

In [24]:
words_wo = w_count(
    df['text'],
    df_cleaning=True,
    stop_words_filtered=True)

'text lowered'

Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

'only words in text '

Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

'text is striped'

'stop words are removed'

In [25]:
words_wo0 = w_count(
    df_non_toxic['text'],
    df_cleaning=True,
    stop_words_filtered = True)

'text lowered'

Pandas Apply:   0%|          | 0/143106 [00:00<?, ?it/s]

'only words in text '

Pandas Apply:   0%|          | 0/143106 [00:00<?, ?it/s]

'text is striped'

'stop words are removed'

In [26]:
words_wo1 = w_count(
    df_non_toxic['text'],
    df_cleaning=True,
    stop_words_filtered=True)

'text lowered'

Pandas Apply:   0%|          | 0/143106 [00:00<?, ?it/s]

'only words in text '

Pandas Apply:   0%|          | 0/143106 [00:00<?, ?it/s]

'text is striped'

'stop words are removed'

In [27]:
def bars(df_list):
    
    fig, ax = plt.subplots(1,3,figsize=(12,8))
    
    ax[0].barh(df_list[0]['Word'],df_list[0]['Frequency'])
    ax[1].barh(df_list[1]['Word'],df_list[1]['Frequency'])
    ax[2].barh(df_list[2]['Word'],df_list[2]['Frequency'])

    

In [134]:
#bars([words_w_trash.head(50),words_w_trash0.head(50),words_w_trash1.head(50)])

In [135]:
#bars([words_wo_trash.head(50),words_wo_trash0.head(50),words_wo_trash1.head(50)])

In [136]:
#bars([words_wo.head(50),words_wo0.head(50),words_wo1.head(50)])

for item in [words_wo_trash,words_wo_trash0,words_wo_trash1]:
    display(item[item['Frequency']>1].tail(50))

for item in [words_w_trash,words_w_trash0,words_w_trash1]:
    display(item[item['Frequency']>1].tail(50))

In [235]:
def nl_split(string):
    
    string = re.split('\n+',string)
    result = [sent_tokenize(i.strip()) for i in string]
        
    return sum(result,[])

In [34]:
df['word_count'] = df['text'].swifter.apply(lambda x: len(word_tokenize(x)))

Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df['sent_count'] = df['text'].swifter.apply(lambda x: len(sent_tokenize(x)))

In [ ]:
df['len'] = df['text'].swifter.apply(lambda x: len(x))

In [ ]:
df[df['toxic']==1]['word_count'].hist(bins=150)

In [ ]:
df[df['toxic']==0]['word_count'].hist(bins=150)

In [ ]:
df[df['toxic']==1]['sent_count'].hist(bins=150)

In [ ]:
df[df['toxic']==0]['sent_count'].hist(bins=150)

In [ ]:
df[df['toxic']==1]['len'].hist(bins=150)

In [ ]:
df[df['toxic']==0]['len'].hist(bins=150)

In [ ]:
def distribution_plt(df_list, name, val_list, plt_xlim=1):

    x_max = min([series.max() for series in df_list])* plt_xlim

    fig, ax = plt.subplots(2,1,figsize=(10,8))
    ax[0].boxplot([df for df in df_list], vert=False)

    ax[0].set_yticklabels(val_list)
    ax[0].set_xlabel(name)
    ax[0].set_xlim([0, x_max])

    ax[1].hist([df for df in df_list], histtype='stepfilled', bins=500)
    ax[1].set_ylabel('частота')
    ax[1].set_xlabel(name)
    ax[1].set_xlim([0, x_max])

    
    plt.suptitle(f'Ящики с усами и гистограмма для признака "{name}"')
    plt.legend(val_list);
    #plt.xlim((0,x_max))

In [ ]:
name_dict = {'word_count' : 'количество слов',
 'sent_count' : 'количество предложений',
 'len' : 'количество символов'}

In [ ]:
for col in ['word_count', 'sent_count', 'len']:
    
    distribution_plt(
        
        [df[df['toxic']==1][col], df[df['toxic']==0][col]],
        
        name_dict[col],
        
        ['токсичные\nкомментарии', 'не токсичные\nкомментарии'],
        
        0.1)

In [ ]:
df.info()

In [ ]:
1/0

In [105]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [36]:
tokenizer

DistilBertTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [37]:
tokenizer.added_tokens_encoder

{'[PAD]': 0, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102, '[MASK]': 103}

In [244]:
def clean_text(string):
    
    
    string = url_extr(string)

    #if len(urls)!=0:
    #    for url in urls:
    #        string = re.sub(url, ' xxlink ', string)

    
    string = re.sub(r'https?://\S+|www\.\S+', ' xxlink ', string) #' [lnkt] '
    string = re.sub(r'<.*?>+', ' xxlink ', string)
    string = re.sub(r'[A-Za-z0-9\_]*@[A-Za-z]*\.?[A-Za-z0-9]*', ' xxmail ', string)

    string = re.sub(r'User:\S+', ' xxuser ', string)
    
    string = re.sub(r'\[{1,2}.+\]{1,2}', ' xxbrct ', string)
    string = re.sub(r'\{{1,2}.+\}{1,2}', ' xxrbrct ', string)
    #string = re.sub('\\d+\\.?', ' dgtl ', string)
    string = re.sub(r'(\W)\1+|(\D)\1+', r'\1', string)
    string = re.sub(r'[\_\+]',' ',string)
    string =  re.sub("['”‘’“]", r'"', string)
    string = re.sub(r'\s+', ' ', string)
    
    return string.strip()#.lower() #best

def clean_text(string):

    string = string.strip()
    #string = re.sub(r'\[?https?://\S+\]?|\[?www\.\S+\]?', ' [lnkt] ', string)

    string = re.sub(r'https?://\S+|www\.\S+', ' lnkt ', string)
    
    string = re.sub(r'\[{1,}.+\]{1,2}', ' brct ', string)
    string = re.sub(r'\{{1,}.+\}{1,}', ' rbrct ', string)
    #string = re.sub('\\d+\\.?', ' dgtl ', string)
    string = re.sub(r'(\W)\1+|(\D)\1+', r'\1', string)
    string = re.sub(r'[\_\+\-]',' ',string)
    
    string = re.sub(r'\s+', ' ', string)
    
    return string.strip()#.lower() #best

def chunker(string):

    sents_ = nl_split(string)

    #tokenized_sents = [tokenizer.encode(clean_text(sent), add_special_tokens=True) for sent in sents_]

    tokenized_sents = [tokenizer.encode(clean_text(sent), add_special_tokens=True) for sent in sents_]

    usents=[]
    usents=tokenized_sents
    
    #for sent in tokenized_sents:
    #    if sent not in usents:# and sent!=[101,1012,102] and sent!=[101,102]:
    #        usents.append(sent)
    
    if len(usents)>1:
       result =  sum(usents, [])
    
    elif len(usents)==1:
        result = usents[0]
    
    else:
        result = tokenizer.encode(sents_[0])

    if len(set(result))/len(string)<0.02:
        result = list(dict.fromkeys(result))
        
    #if len(result)>512:
    #    result = result[:511]+[102]
    #    result = [101] + result[:511]+[102]
        #result = result[:129] + result[-382:]+[102]
        
    
    return [101] + result[:510]+[102]#result

In [249]:
def chunker(string):

    sents_ = nl_split(string)

    sents_clean = [clean_text(sent) for sent in  sents_]

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_clean]

    #for sent in tokenized_sents:
    #    if sent not in usents:# and sent!=[101,1012,102] and sent!=[101,102]:
    #        usents.append(sent)

    if len(tokenized_sents)>1:
        result = sum(tokenized_sents,[])
    else:
        result = tokenized_sents[0]
    
        
    if len(result)>512:

        result = result[:129] + result[-382:]+[102]
        
    
    return result

In [250]:
chunker(df['text'].sample())

TypeError: expected string or bytes-like object, got 'list'

tokenizer.decode(chunker(df['text'][96905]))

tokenizer.decode(chunker(df['text'][4704]))

tokenizer.decode(chunker(df['text'][82496]))

df['first_sent'] = df['text'].swifter.apply(lambda x: nl_split(x)[0])

df['after_tok'] = df['text'].swifter.apply(lambda x: tokenizer.decode(chunker(x)))

df['a_t_len'] = df['after_tok'].apply(lambda x: len(x))

df['after_tok'].duplicated().sum()

orientation_list = ['gay', 'lesb', 'trans', 'lgbt']
relig_list = ['christ', 'islam', 'judai','budd', 'jesus', 'allah', 'yahweh']
family_list = ['mom', 'dad', 'papa', 'daddy', 'son', 'daughter', 'mother', 'child']

df['orient'] =  df['text'].str.contains('|'.join(orientation_list), flags=re.IGNORECASE)
df['relig'] =  df['text'].str.contains('|'.join(relig_list), flags=re.IGNORECASE)
df['family'] =  df['text'].str.contains('|'.join(family_list), flags=re.IGNORECASE)

df.sort_values(by='after_tok').iloc[50:100]

df.drop_duplicates(subset='after_tok', inplace=True)

df.reset_index(drop=True, inplace=True)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [186]:
def logy(feats, target):
    lr = LogisticRegression(random_state=RANDOM_STATE,max_iter=100000,class_weight='balanced')
    lr.fit(feats, target)
    scores = cross_val_score(lr, feats, target, cv=5, scoring='f1_macro')
    return scores

In [ ]:
1/0

In [ ]:
df.shape[0]//50

In [ ]:
3185*50

In [187]:
samp = df.sample(n=32000, random_state=RANDOM_STATE)

In [43]:
159200*0.2

31840.0

In [188]:
X_train, X_test, y_train, y_test = train_test_split(
    #samp[['text','orient','relig','family']],
    samp['text'],
    samp['toxic'],
    test_size = 0.5,
    random_state = RANDOM_STATE,
    stratify = samp['toxic'])

In [189]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16000,), (16000,), (16000,), (16000,))

sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 

X_resample.shape, y_resample.shape

res_df = X_resample.join(y_resample)

res_samp = res_df.sample(n=25000, random_state=RANDOM_STATE)

test_samp = pd.DataFrame(X_test).join(pd.DataFrame(y_test)).sample(n=res_samp.shape[0], random_state=RANDOM_STATE)

res_samp.shape

test_samp.shape

In [190]:
class BertTokenizer(object):

    def __init__(self, text, batch_size=1):
        
        self.text = text
        self.batch_size = batch_size
        #self.model_class ,self.pretrained_weights = (
        #    transformers.BertModel, 'bert-base-uncased')
        self.model_class ,self.pretrained_weights = (
            transformers.DistilBertModel, 'distilbert-base-uncased')    
        
        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    
    def get(self):
        
        tokenized = self.text.swifter.apply(lambda x: chunker(x))
        
        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)

        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(len(padded) // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features

df[df['text'].str.contains('I ass. I ass.')]['text']

tokenizer.decode(chunker(df['text'][4703]))

tokenizer.decode(chunker(df['text'][0]))

tokenizer.decode(chunker('someone needs to remove lott 71. 227. 167. 147'))

In [47]:
BertTokenizer(df['text'][0:2]).get()

Pandas Apply:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

array([[ 0.11305679, -0.02071994,  0.11411636, ...,  0.08299962,
         0.27643913,  0.33111763],
       [-0.13494527, -0.0798372 ,  0.2560554 , ..., -0.03249058,
         0.27541596,  0.4066586 ]], dtype=float32)

In [ ]:
16000/32

In [69]:
clean_text(df['text'][5327])

"Don't vandalize other wikipedians pages. 2nd warning. link"

In [642]:
dembs_train = BertTokenizer(text=X_train, batch_size=32).get()

Pandas Apply:   0%|          | 0/16000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [653]:
logy(dembs_train, y_train.values).mean()

np.float64(0.8099593303107095)

In [73]:
logy(dembs_train, y_train.values).mean()

0.8073131369641621

In [654]:
dembs_test = BertTokenizer(text=X_test, batch_size=32).get()

Pandas Apply:   0%|          | 0/16000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
#tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#mod = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
###############

In [ ]:
def mean_embs(string, model ):
    sents_ = nl_split(string)

    sents_ = [clean_text(sent) for sent in sents_]
    
    tokens = tokenizer(sents_,max_length=512, padding=True, truncation=True, return_tensors='np')
    #model = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')
    with torch.no_grad():
        batch_embeddings = model(torch.LongTensor(tokens['input_ids']), attention_mask=torch.LongTensor(tokens['attention_mask']))

    embs = batch_embeddings[0][:,0,:].numpy()
    
    
    return np.mean(embs, axis=0)#.reshape(1,768)

In [ ]:
pd.DataFrame(dembs_train).duplicated().sum()

In [75]:
np.array([0.80239492, 0.7986666 , 0.8033005 , 0.79858842, 0.81225364]).mean()

0.8030408159999999

In [ ]:
#array([0.8919973 , 0.8999975 , 0.90249588, 0.89397286, 0.89449786])

In [ ]:
###dembs_test = BertTokenizer(text=test_samp['text'], batch_size=50).get()
####logy(dembs_train, res_samp['toxic'])

In [ ]:
#dembs_test = BertTokenizer(text=X_test['text'], batch_size=50).get()

In [ ]:
#tr = pd.DataFrame(trial).join(pd.DataFrame(res_samp['toxic'].reset_index(drop=1)))

In [ ]:
#tr.head().iloc[:,-5:]

In [ ]:
#tr.tail().iloc[:,-5:]

In [ ]:
#tr.drop_duplicates(inplace=True)
#tr.reset_index(inplace=True,drop=True)

In [ ]:
#tr.tail().iloc[:,-5:]

from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train 
)

sample_weights

In [655]:
def f1_eval(y_true, y_pred,):

    f1 = f1_score(y_pred, y_true, average=None)[1]
    return f1

In [656]:
def f1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    f1 = f1_score( y_pred,y_true, average=None)[1]

    return 'f1', f1

In [657]:
from sklearn.metrics import make_scorer
my_func = make_scorer(f1_eval, greater_is_better=True)

In [658]:
pipe = Pipeline([('models',[LogisticRegression(random_state=RANDOM_STATE,class_weight='balanced')])])

In [659]:
p_wei = y_train.value_counts()[0]/y_train.value_counts()[1]

In [726]:
param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        #'models__C': [0.5,1,3,5],#,
        #'models__penalty': ['l1', 'l2']
          
    },

    {
        'models': [XGBClassifier(
            random_state=RANDOM_STATE,
            ###objective = 'binary:logistic',
            eval_metric = f1_xgb,
            #maximize=True,
            #eval_metric='logloss',
            #scale_pos_weight = y_train.value_counts(normalize=1)[0]
            scale_pos_weight = p_wei
            
            )],
        'models__n_estimators':[500, 1000, 1250],#[1000, 1250],#, 1500, 2000],#[1000, 1250, 1500, 2000],
        'models__max_depth':[4],#[4,5,6],
        'models__learning_rate':[0.3,0.2, 0.1, 0.4]#[0.1,0.2, 0.25 , 0.3],[0.3,0.4,0.2, 0.1, 0.05]
        #'models__alpha':[0.1, 0.3]#,
        #'models__eval_metric': [f1_xgb]
    }

]

In [661]:
y_train.value_counts(normalize=1)

toxic
0    0.896813
1    0.103188
Name: proportion, dtype: float64

In [662]:
y_train.value_counts()[0]/y_train.value_counts()[1]

np.float64(8.691096305269534)

param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        'models__C': [0.5,1,3,5],
        #'models__penalty': ['l1', 'l2']
          
    }
]

In [727]:
grid_search = GridSearchCV(
    pipe, 
    param_grid, 
    cv=5,
    #scoring='f1',
    scoring=my_func,
    n_jobs=-1
)
grid_search.fit(dembs_train, y_train)
#grid_search.fit(trial, y_train.values)
#grid_search.fit(trial, res_samp['toxic'])
###grid_search.fit(tr.drop('toxic', axis=1), tr['toxic'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('models',
                                        [LogisticRegression(class_weight='balanced',
                                                            random_state=88)])]),
             n_jobs=-1,
             param_grid=[{'models': [LogisticRegression(class_weight='balanced',
                                                        random_state=88)],
                          'models__solver': ['newton-cg', 'liblinear']},
                         {'models': [XGBClassifier(base_score=None,
                                                   booster=None, callbacks=None,
                                                   colsample_bylevel=None,
                                                   c...
                                                   max_delta_step=None,
                                                   max_depth=None,
                                                   max_leaves=None,
                                                   min_child_weight=None,
                                                   missing=nan,
                                                   monotone_constraints=None,
                                                   multi_strategy=None,
                                                   n_estimators=None,
                                                   n_jobs=None,
                                                   num_parallel_tree=None, ...)],
                          'models__learning_rate': [0.3, 0.2, 0.1, 0.4],
                          'models__max_depth': [4],
                          'models__n_estimators': [500, 1000, 1250]}],
             scoring=make_scorer(f1_eval, response_method='predict'))

In [ ]:
#n_estimators 	2000, learning_rate 	0.1, max_depth 	5

In [728]:
pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'mean_test_score','params']].sort_values('rank_test_score')

,rank_test_score,mean_test_score,params
5,1,0.712537,"{'models': XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None, device=None, early_stopping_rounds=None, enable_categorical=False, eval_metric=<function f1_xgb at 0x000002541A5DECA0>, feature_types=None, feature_weights=None, gamma=None, grow_policy=None, importance_type=None, interaction_constraints=None, learning_rate=None, max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None, max_depth=None, max_leaves=None, min_child_weight=None, missing=nan, monotone_constraints=None, multi_strategy=None, n_estimators=None, n_jobs=None, num_parallel_tree=None, ...), 'models__learning_rate': 0.2, 'models__max_depth': 4, 'models__n_estimators': 500}"
6,2,0.710526,"{'models': XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None, device=None, early_stopping_rounds=None, enable_categorical=False, eval_metric=<function f1_xgb at 0x000002541A5DECA0>, feature_types=None, feature_weights=None, gamma=None, grow_policy=None, importance_type=None, interaction_constraints=None, learning_rate=None, max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None, max_depth=None, max_leaves=None, min_child_weight=None, missing=nan, monotone_constraints=None, multi_strategy=None, n_estimators=None, n_jobs=None, num_parallel_tree=None, ...), 'models__learning_rate': 0.2, 'models__max_depth': 4, 'models__n_estimators': 1000}"
7,3,0.709485,"{'models': XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None, device=None, early_stopping_rounds=None, enable_categorical=False, eval_metric=<function f1_xgb at 0x000002541A5DECA0>, feature_types=None, feature_weights=None, gamma=None, grow_policy=None, importance_type=None, interaction_constraints=None, learning_rate=None, max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None, max_depth=None, max_leaves=None, min_child_weight=None, missing=nan, monotone_constraints=None, multi_strategy=None, n_estimators=None, n_jobs=None, num_parallel_tree=None, ...), 'models__learning_rate': 0.2, 'models__max_depth': 4, 'models__n_estimators': 1250}"
2,4,0.708585,"{'models': XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None, device=None, early_stopping_rounds=None, enable_categorical=False, eval_metric=<function f1_xgb at 0x000002541A5DECA0>, feature_types=None, feature_weights=None, gamma=None, grow_policy=None, importance_type=None, interaction_constraints=None, learning_rate=None, max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None, max_depth=None, max_leaves=None, min_child_weight=None, missing=nan, monotone_constraints=None, multi_strategy=None, n_estimators=None, n_jobs=None, num_parallel_tree=None, ...), 'models__learning_rate': 0.3, 'models__max_depth': 4, 'models__n_estimators': 500}"
3,5,0.706557,"{'models': XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None, device=None, early_stopping_rounds=None, enable_categorical=False, eval_metric=<function f1_xgb at 0x000002541A5DECA0>, feature_types=None, feature_weights=None, gamma=None, grow_policy=None, importance_type=None, interaction_constraints=None, learning_rate=None, max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None, max_depth=None, max_leaves=None, min_child_weight=None, missing=nan, monotone_constraints=None, multi_strategy=None, n_estimators=None, n_jobs=None, num_parallel_tree=None, ...), 'models__learning_rate': 0.3, 'models__max_depth': 4, 'models__n_estimators': 1000}"
4,6,0.706455,"{'models': XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=None, colsample_bynode=

In [ ]:
pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'mean_test_score','params']].sort_values('rank_test_score').iloc[2]['params']#.head(16)

In [ ]:
#grid_pred = grid_search.best_estimator_.predict(debs_no_dups_test.drop('toxic', axis=1))

In [729]:
#grid_pred = grid_search.best_estimator_.predict(trial_trial)
grid_pred = grid_search.best_estimator_.predict(dembs_test)

In [730]:
f1_score(grid_pred, y_test,average=None)

array([0.96988348, 0.71711595])

In [731]:
print(classification_report(grid_pred, y_test))
#print(classification_report(grid_pred, test_samp['toxic']))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     14572
           1       0.67      0.77      0.72      1428

    accuracy                           0.95     16000
   macro avg       0.82      0.87      0.84     16000
weighted avg       0.95      0.95      0.95     16000



In [733]:


classifier_tuned_threshold = TunedThresholdClassifierCV(
    grid_search.best_estimator_, scoring=my_func, n_jobs=-1
).fit(dembs_train, y_train)


In [734]:
ctt_pred = classifier_tuned_threshold.predict(dembs_test)

In [736]:
classifier_tuned_threshold.best_threshold_

np.float32(0.2020202)

In [735]:
f1_score(y_test, ctt_pred)

0.7221570926143025

In [719]:
classifier_other_threshold = FixedThresholdClassifier(
    grid_search.best_estimator_, threshold=0.35, response_method="predict_proba"
).fit(dembs_train, y_train)

cot_pred = classifier_other_threshold.predict(dembs_test)

f1_score(y_test, cot_pred)

0.722735674676525

In [ ]:
classifier_other_threshold = FixedThresholdClassifier(
    grid_search.best_estimator_, threshold=0.6, response_method="predict_proba"
).fit(debs_no_dups.drop('toxic', axis=1), debs_no_dups['toxic'])
f1_score(debs_no_dups_test['toxic'], classifier_other_threshold.predict(debs_no_dups_test.drop('toxic', axis=1)))#,average=None)#82

In [ ]:
pd.DataFrame(dembs_train).duplicated().sum()

In [ ]:
f1_score(y_test, classifier_other_threshold.predict(dembs_test))#82

In [ ]:
cl_preds_adj = classifier_tuned.predict(dembs_val)

In [ ]:
f1_score(cl_preds_adj,y_val)

In [ ]:
res =pd.DataFrame({'a':grid_pred, 'b':test_sample['toxic']})

In [ ]:
pd.DataFrame(probs[0])

In [ ]:
y_test.shape

In [ ]:
pd.DataFrame({'proba':list(probs[1])},
              {'predicted':grid_pred})

In [ ]:
probs_res = pd.DataFrame(zip(y_test,probs[1],grid_pred),columns=['toxic', 'proba', 'pred'])

In [ ]:
probs_res[(probs_res['toxic']!=probs_res['pred']&(probs_res['toxic']==1))]['proba']

In [ ]:
X_test.iloc[1961]

In [ ]:
X_train.apply(lambda x: len(x)).hist()

In [ ]:
X_train.shape

In [ ]:
X_resample.shape

In [ ]:
X_train[X_train.apply(lambda x: len(x))>2000]

In [ ]:
res.shape

In [ ]:
df['text'].

In [ ]:
df[(df.index.isin(res[res['a']!=res['b']].index))&(df['toxic']==1)]['text'].apply(lambda x: tokenizer.decode(chunker(x)))#.sample()



#['b'].value_counts()

In [ ]:
tokenizer.decode(chunker(df.loc[128933]['text']))

In [ ]:
df.loc[128933]['text']

## Обучение

In [ ]:
#from phik.report import plot_correlation_matrix
#from phik import phik_matrix

In [ ]:
#df.info()

phik_overview = phik_matrix(df[['toxic','word_count']],verbose=False)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Blues',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(18, 14)
) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_lemm'], 
    df['toxic'], 
    random_state=RANDOM_STATE,
    stratify=df['toxic']
)

In [ ]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 


In [ ]:
y_resample.value_counts()

In [ ]:
pd.DataFrame(y_test).value_counts()

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны